In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

from scipy.io import loadmat
from scipy.misc import imread

import os, pickle

from util import *
%load_ext autoreload
%aimport util
%autoreload 1

In [ ]:
videos = {}
squat_dir = Data + 'squats/'
for video in os.listdir(squat_dir):
    frames = []
    video_dir = squat_dir + video + '/'
    for frame in sorted(os.listdir(video_dir)):
        path = video_dir + str(frame)
        frames.append(imread(path))
    videos[video] = np.array(frames)

In [32]:
names = sorted(videos.keys())
np.random.seed(0)
np.random.shuffle(names)

In [34]:
N = len(names)
train_indices = int(0.8 * N)
val_indices = int(0.1 * N)
test_indices = int(0.1 * N)
dataset_names = {
    'train' : names[: train_indices],
    'val' : names[train_indices : train_indices + val_indices],
    'test' : names[train_indices + val_indices :]
}

In [2]:
labels_L_1 = {}
squat_label_dir = Data + 'squat_labels/'
for video in os.listdir(squat_label_dir):
    frames = []
    video_dir = squat_label_dir + video + '/'
    for frame in sorted(os.listdir(video_dir)):
        file = video_dir + frame
        frames.append(imread(file).reshape((224, 224, 1)))
    labels_L_1[video] = np.array(frames)

In [42]:
for set_name, video_names in dataset_names.items():
    set_dir = output_dir + set_name + '/'
    make_dir(set_dir)
    for name in video_names:
        video_dir = set_dir + name
        make_dir(video_dir)
        np.save(video_dir + '/video', videos[name])
        np.save(video_dir + '/poses_L_1', labels_L_1[name])

In [30]:
labels_L_13 = {}
squat_label_multiple_dir = Data + 'squats_labels_multiple/'
for video in os.listdir(squat_label_multiple_dir):
    frames = []
    video_dir = squat_label_multiple_dir + video + '/'
    for frame in sorted(os.listdir(video_dir)):
        frame_dir = video_dir + frame + '/'
        pose_stack = []
        for file in sorted(os.listdir(frame_dir), key=lambda x: int(x[:-4])):
            pose_stack.append(imread(frame_dir + file).reshape((224, 224, 1)))
        frames.append(np.concatenate(pose_stack, axis=2))
    labels_L_13[video] = np.array(frames)